In [36]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed
from listUtils import getFlatList
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs, findNearest, findAll
from fileUtils import getDirBasics, getBaseFilename
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir, removeDir
from pandasUtils import getRowDataByColValue, getRowData
import operator
from glob import glob
from os.path import join
from collections import Counter


################################################################################
## Mp3 Stuff
################################################################################
from mp3id import mp3ID


################################################################################
## Database Stuff
################################################################################
from discogsBase import discogs
from mainDB import getAlbumTypes
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import getArtistAlbumsDB, discConv


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap
from musicBase import myMusicBase
from musicMatch import myMatchedMusic
from mergeDB import searchForMutualDBEntries, searchForMutualArtistDBEntries


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-04-26 17:46:59.232160


# My Music Database Map

In [40]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()

Current Time is Sun Apr 26, 2020 18:15:18 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Sun Apr 26, 2020 18:15:56 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 37 seconds.
Summary Statistics For DB: Discogs
    Found 736858 ID -> Name entries
    Found 675148 Name -> ID entries
    Found 0 Albums
Current Time is Sun Apr 26, 2020 18:15:57 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Sun Apr 26, 2020 18:15:58 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
==================================

# Music Base

In [41]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Sun Apr 26, 2020 18:16:05 for Finding All Artist Albums
Current Time is Sun Apr 26, 2020 18:16:47 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 41 seconds.


# Create My Matched Music Artists

In [42]:
mmm = myMatchedMusic(mdb)

My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']


## Get Artist Status and Find Unknowns

In [43]:
mmm.getArtistStatus()
mmm.setMusicBase(mmb)
mmm.getUnknownArtists()

Current Time is Sun Apr 26, 2020 18:16:47 for Matching All Music Artists
Current Time is Sun Apr 26, 2020 18:16:47 for Done with Matching All Music Artists
Process [Done with Matching All Music Artists] took 128.0 millseconds.
Found 1 unknown artists
Found 0 total artists


{"Killin' Baudelaire": ["/Volumes/Biggy/Matched/K/Killin' Baudelaire"]}

***
***

# Match Albums From My Matched Artists

In [49]:
dR = 0.1
rC = 0.9

mmm.matchMyMusicAlbums(db="AllMusic", albumType=1, ratioCut=rC, maxCut=rC+dR)

Current Time is Sun Apr 26, 2020 18:21:38 for Checking for Albums Matches Against AllMusic DB
Artist                                  Database       Album Name                                    --> Matched Album
Artist ID [0000691561] is not found in Albums DB [AllMusic]
Artist ID [0000112185] is not found in Albums DB [AllMusic]
Rascal Flatts                           AllMusic       Still Feels Good                              --> {'Name': 'Still Feels Good', 'Code': '0000780122', 'MediaType': 'Albums'}
Current Time is Sun Apr 26, 2020 18:21:52 for Done with Checking for Albums Matches Against AllMusic DB
Process [Done with Checking for Albums Matches Against AllMusic DB] took 14 seconds.
Saving data to myMusicAlbumMatch.yaml
Saved data to myMusicAlbumMatch.yaml
  --> This file is 211B.
Found 1 music <-> discogs albums maps


In [50]:
mmm.moveMyMatchedMusicAlbums(show=False)

Found 1 music <-> discogs albums maps
==> Rascal Flatts
/Volumes/Music/Matched/R/Rascal Flatts/Still Feels Good does not exist


In [48]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Music/Matched', '/Volumes/Biggy/Matched']
Current Time is Sun Apr 26, 2020 18:20:55 for Finding All Artist Albums
Current Time is Sun Apr 26, 2020 18:21:35 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 39 seconds.


***
***

# UnMatch Artist

In [ ]:

def unMatchArtist(artistName):
    from os.path import join
    from fsUtils import removeDir, isFile, setFile, removeFile

    for musicDir in getMatchedDirs():
        dirval        = join(musicDir, getPrimeDirectory(artistName), artistName)
        if not isDir(dirval):
            continue
            
        matchedDir    = setDir(dirval, "Match")
        mediaTypeDirs = findDirs(matchedDir)
        for mediaTypeDir in mediaTypeDirs:
            for matchDir in findDirs(mediaTypeDir):
                albumName = getUnMatchedDirName(getDirBasics(matchDir)[-1], mediaTypeDir)

                srcDir = matchDir
                dstDir = setDir(dirval, albumName)
                if isDir(dstDir):
                    i = 0
                    while not isDir(dstDir):
                        dstDir = "{0} [Fix-{1}]".format(setDir(dirval, albumName), i)
                        i += 1

                moveDir(srcDir, dstDir, debug=True)

            if isDir(mediaTypeDir):
                DS_Store = setFile(mediaTypeDir, ".DS_Store")
                if isFile(DS_Store):
                    removeFile(DS_Store, debug=True)
                removeDir(mediaTypeDir, debug=True)

        if isDir(matchedDir):
            DS_Store = setFile(matchedDir, ".DS_Store")
            if isFile(DS_Store):
                removeFile(DS_Store, debug=True)
            removeDir(matchedDir, debug=True)
            

In [ ]:
unMatchArtist("The J. Geils Band")

In [ ]:
######################################################################
#### Loop Over Prime Directories
######################################################################
for primeDir in getPrimeDirectories():
    artistPrimeDirMap = getArtistPrimeDirMap(primeDir)

    ######################################################################
    #### Loop Over Artist Name <-> Prime Map Items
    ######################################################################
    for artistName, artistPrimeDirs in artistPrimeDirMap.items():
        unMatchArtist(artistName)


***
***

# Merge DBs After Finding Matches

In [37]:
retval = searchForMutualDBEntries(mdb, minI=-1, cutoff=0.7, maxR=3000)

Searching for matches:  [A Flock Of Seagulls] using [230] albums collected from [7] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'DatPiff', 'MusicStack', 'MetalStorm']
AceBootlegs         	
DatPiff             	
MusicStack          	
MetalStorm          	
Searching for matches:  [A Tribe Called Quest] using [120] albums collected from [5] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [ABBA] using [515] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matc

DatPiff             	
RockCorner          	
MusicStack          	
MetalStorm          	
Searching for matches:  [Anal Cunt] using [72] albums collected from [6] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
MetalStorm          	
Searching for matches:  [Ancient] using [19] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'LastFM', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [And Oceans] using [22] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'LastFM', 'CDandLP', 'MusicStack', 'Met

RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Awolnation] using [111] albums collected from [6] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
MusicStack          	
MetalStorm          	
Searching for matches:  [B.B. King] using [1073] albums collected from [6] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Backstreet Boys] using [238] albums collected from [5] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
MusicStack  

DatPiff             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Black Dice] using [75] albums collected from [5] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
MusicStack          	
MetalStorm          	
Searching for matches:  [Black Eyed Peas] using [180] albums collected from [6] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'RockCorner', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
RockCorner          	
MusicStack          	
MetalStorm          	
Searching for matches:  [Black Sabbath] using [429] albums collected from [7] dbs
  Will search for matches in these DBs: ['RateYourMusic', 'CDandLP', 'MusicStack', 'MetalStorm']
RateYourMusic       	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:

AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Bluegrass Etc.] using [5] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Blueline Medic] using [11] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Blues Saraceno] using [31] albums collected from [4] dbs
  Will search for m

...

DatPiff             	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Real McCoy] using [55] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'LastFM', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Rise Against] using [133] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'LastFM', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Walter Becker] using [35] albums collected from [3] dbs
  Will search for matches in these DBs: 

LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Gavin DeGraw] using [44] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'DatPiff', 'RockCorner', 'LastFM', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
DatPiff             	
RockCorner          	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Jordin Sparks] using [39] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'RockCorner', 'LastFM', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
RockCorner          	
LastFM              	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Toni Braxton & Babyface] using [167] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceB

DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Doja Cat] using [16] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Dua Lipa] using [62] albums collected from [2] dbs
  Will search for matches in these DBs: ['AllMusic', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AllMusic            	
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Gloria Gaynor] using [346] albums

AllMusic            	
MusicBrainz         	
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Spencer Davis Group] using [204] albums collected from [3] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [St. Vincent] using [79] albums collected from [4] dbs
  Will search for matches in these DBs: ['AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'CDandLP', 'MusicStack', 'MetalStorm']
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
CDandLP             	
MusicStack          	
MetalStorm          	
Sea

LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [Goldfrapp] using [108] albums collected from [2] dbs
  Will search for matches in these DBs: ['MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
MusicBrainz         	
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
MusicStack          	
MetalStorm          	
Searching for matches:  [He Is Legend] using [8] albums collected from [1] dbs
  Will search for matches in these DBs: ['AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
AllMusic            	
MusicBrainz         	
AceBootlegs         	
RateYourMusic       	
LastFM              	
DatPiff             	
RockCorner          	
CDandLP             	
M

In [38]:
retval

{'Johnny Winter': {'CDandLP': {'ID': '27962339377189', 'Name': None}},
 'Larry Williams': {'CDandLP': {'ID': '22696137688516', 'Name': None}},
 'The Call': {'AceBootlegs': {'ID': '806844302417', 'Name': None}},
 'The Supremes': {'AceBootlegs': {'ID': '543243890628', 'Name': None}},
 'The Seeds': {'AceBootlegs': {'ID': '224179138861', 'Name': None},
  'RateYourMusic': {'ID': '8', 'Name': None}},
 'The Crows': {'AceBootlegs': {'ID': '543243890628', 'Name': None}},
 'The Mixtures': {'AceBootlegs': {'ID': '543243890628', 'Name': None}},
 'The Wailers': {'AceBootlegs': {'ID': '224179138861', 'Name': None}},
 'Kevn Kinney': {'AllMusic': {'ID': '0000769385', 'Name': None}},
 'Willie Dixon': {'MusicBrainz': {'ID': '92595780574399326938241774884809702708',
   'Name': None}},
 'John Waite': {'CDandLP': {'ID': '27962339377189', 'Name': None}}}

In [39]:
if len(retval) > 0:
    for artistName,artistResult in retval.items():
        for db,dbResult in artistResult.items():
            mdb.add(artistName, db, dbResult["ID"])
    mdb.save()

Adding Database [CDandLP] to DB list for [Johnny Winter]
Artist DB Data: {'Discogs': {'ID': '254198', 'Name': None}, 'AllMusic': {'ID': '0000819983', 'Name': None}, 'MusicBrainz': {'ID': '44101796377774413253903454212584419888', 'Name': None}, 'AceBootlegs': None, 'LastFM': {'ID': '23062638876', 'Name': None}, 'RateYourMusic': None, 'DatPiff': None, 'RockCorner': {'ID': '764258996', 'Name': None}, 'CDandLP': {'ID': '27962339377189', 'Name': None}, 'MusicStack': None, 'MetalStorm': None}
Adding Database [CDandLP] to DB list for [Larry Williams]
Artist DB Data: {'Discogs': {'ID': '553067', 'Name': None}, 'AllMusic': {'ID': '0000135024', 'Name': None}, 'MusicBrainz': {'ID': '280274669648859583306482668754534362685', 'Name': None}, 'AceBootlegs': None, 'RateYourMusic': None, 'LastFM': {'ID': '74503802032', 'Name': None}, 'DatPiff': None, 'RockCorner': {'ID': '79340108', 'Name': None}, 'CDandLP': {'ID': '22696137688516', 'Name': None}, 'MusicStack': None, 'MetalStorm': None}
Adding Database